<a href="https://colab.research.google.com/github/pri2k/ML-Practice-Repositories/blob/main/English%20Nagamese%20Language%20Translation/Eng_Ngm_M2M_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

belovedorange_english_nagamese_raw_data_path = kagglehub.dataset_download('belovedorange/english-nagamese-raw-data')

print('Data source import complete.')


In [ ]:
!pip install torch torchvision transformers

In [ ]:
import pandas as pd

# Read the CSV file
data = pd.read_csv("/kaggle/input/english-nagamese-raw-data/raw_data.csv")

# Verify the data
print(data.head())


                                             English  \
0  Paul, Silvanus, and Timothy to the church of t...   
1  We always give thanks to God for all of you as...   
2  We remember before our God and Father your wor...   
3  Brothers loved by God, we know he has chosen you,   
4  because our gospel came to you not in word onl...   

                                            Nagamese  
0  Paul aru Silvanus aru Timothy pora Isor aru Pr...  
1  Amikhan hodai apnikhan nimite Isor ke dhanyaba...  
2  Amikhan pora apni khan laga biswas laga kaam, ...  
3  Isor pora morom kora bhai khan, amikhan jane T...  
4  kilemane Isor laga kotha apni khan logote khal...  


In [ ]:
# data=data[:100]

In [ ]:
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Load model and tokenizer
model_name = "facebook/m2m100_418M"
model = M2M100ForConditionalGeneration.from_pretrained(model_name)
tokenizer = M2M100Tokenizer.from_pretrained(model_name)



In [ ]:
# Prepare dataset
source_texts = list(data['English'])
target_texts = list(data['Nagamese'])

# Tokenize the data
inputs = tokenizer(source_texts, return_tensors="pt", max_length=128, truncation=True, padding="max_length")
labels = tokenizer(target_texts, return_tensors="pt", max_length=128, truncation=True, padding="max_length")["input_ids"]

# Adjust labels for loss calculation
labels[labels == tokenizer.pad_token_id] = -100

# Create DataLoader
from torch.utils.data import DataLoader, TensorDataset

dataset = TensorDataset(inputs["input_ids"], inputs["attention_mask"], labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Fine-tuning loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(128112, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(128112, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100SdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
   

In [ ]:
epochs = 3
for epoch in range(epochs):
    model.train()
    i = 0
    for batch in dataloader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print("iterating ", i)
        i = i+1
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# Save the fine-tuned model
model.save_pretrained("./m2m100-finetuned-english-nagamese")
tokenizer.save_pretrained("./m2m100-finetuned-english-nagamese")

iterating  0
iterating  1
iterating  2
iterating  3
iterating  4
iterating  5
iterating  6
Epoch 1, Loss: 5.061554431915283
iterating  0
iterating  1
iterating  2
iterating  3
iterating  4
iterating  5
iterating  6
Epoch 2, Loss: 3.959017515182495
iterating  0
iterating  1
iterating  2
iterating  3
iterating  4
iterating  5
iterating  6
Epoch 3, Loss: 3.9644460678100586


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('./m2m100-finetuned-english-nagamese/tokenizer_config.json',
 './m2m100-finetuned-english-nagamese/special_tokens_map.json',
 'm2m100-finetuned-english-nagamese/vocab.json',
 'm2m100-finetuned-english-nagamese/sentencepiece.bpe.model',
 './m2m100-finetuned-english-nagamese/added_tokens.json')